In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
import plotly.express as px

from sklearn.model_selection import train_test_split #Parte la base de datos (80-20)
from imblearn.over_sampling import SMOTE #Para balancear una base de datos, uno de los metodos para crear datos con los que ya evisten es SMOTE (Synthetico Oversampling algo Technique)

from sklearn.linear_model import LogisticRegression #Ajusta los datos a una curva logísdtica y toma decisiones sobre eso. 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelEncoder



from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,classification_report #Para incluir la curva de ROC
df = pd.read_csv('BDUA.csv')

Se eliminarán: 
* Código de la entidad: es solo un sistema de codificación interna que traduce el nombre de la entidad por lo que solo es necesario conservar una de las columnas. Para este caso conservaremos la de nombre de la entidad ya que ofrece información más fácil de interpretar.
* Régimen: todos son de orden contributivo.
* Municipio: son demasiados datos y muy diferentes. Además, ya se cuenta con un filtro geográfico que es el departamento.
* Nivel del Sisbén: hay mucha diversidad de datos de los cuales son faltantes el 86% de los datos. Además, ya se cuenta con un filtro relacionado con el sisbén que es el nivel. 
* Cantidad de registros: solo enumera las veces que se han validado los datos que se muestraa. No realiza ningun aporte con la clasificación de EPS. 

<br></br>
*Eliminación de columnas*

In [48]:
columnas_Eliminar = ['Código de la entidad', 'Régimen', 'Municipio', 'Nivel del Sisbén', 'Cantidad de registros']

df_drop = df.drop(columns=columnas_Eliminar)
df_drop.head()

,Género,Grupo etario,Nombre de la entidad,Tipo de afiliado,Estado del afiliado,Condición del beneficiario,Zona de Afiliación,Departamento,tps_nvl_ssb_id
0,Masculino,19 a 45,ASMET SALUD EPS S.A.S. -CM,COTIZANTE,Activo,NO APLICA,Rural,CAUCA,N
1,Femenino,50 a 55,EPS FAMISANAR S.A.S.,COTIZANTE,Activo,NO APLICA,Urbana,TOLIMA,NaN
2,Femenino,5 a 15,COOSALUD EPS S.A. -CM,BENEFICIARIO,Activo,NO APLICA,Urbana,ANTIOQUIA,NaN
3,Femenino,15 a 19,ENTIDAD PROMOTORA DE SALUD SANITAS S.A.S.,COTIZANTE,Activo,NO APLICA,Urbana,CAUCA,NaN
4,Masculino,70 a 75,EPS FAMISANAR S.A.S.,BENEFICIARIO,Activo,NO APLICA,Urbana,CUNDINAMARCA,NaN


<br></br>
*Codificación de variables tipo objeto*</br>
Ahora, hay que hacer la codificación para convertir los objetos a variables numéricas. Se hará codificación One-hot y codificación de etiquetas para diferentes columnas. 

In [50]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

columnas_one_hot = [ 'Tipo de afiliado', 'Estado del afiliado', 'Condición del beneficiario', 'Zona de Afiliación', 'Género']
columnas_etiquetas = ['Grupo etario','Nombre de la entidad', 'Departamento', 'tps_nvl_ssb_id']


datos_codificados_one_hot = pd.get_dummies(df_drop[columnas_one_hot])

# Codificación de etiquetas
datos_codificados_etiquetas = pd.DataFrame()
label_encoder = LabelEncoder()
for columna in columnas_etiquetas:
    datos_codificados_etiquetas[columna + '_codificado'] = label_encoder.fit_transform(df_drop[columna])



columnas_no_codificadas = [col for col in df_drop.columns if col not in columnas_one_hot + columnas_etiquetas]
datos_no_codificados = df_drop[columnas_no_codificadas]

datos_codificados = pd.concat([datos_codificados_one_hot, datos_codificados_etiquetas, datos_no_codificados], axis=1)
datos_codificados = datos_codificados.replace({False: 0, True: 1})
mapeos = {}

label_encoder = LabelEncoder()
for columna in columnas_etiquetas:
    df_drop[columna + '_codificado'] = label_encoder.fit_transform(df_drop[columna])
    mapeo = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    mapeos[columna] = mapeo

# Mostrar los mapeos
print("Mapeo entre valores originales y codificados:")
for columna, mapeo in mapeos.items():
    print(f"{columna}: {mapeo}")

C:\Users\LauraG\AppData\Local\Temp\ipykernel_23492\4125068818.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos_codificados = datos_codificados.replace({False: 0, True: 1})


Mapeo entre valores originales y codificados:
Grupo etario: {'1 a 5': 0, '15 a 19': 1, '19 a 45': 2, '45 a 50': 3, '5 a 15': 4, '50 a 55': 5, '55 a 60': 6, '60 a 65': 7, '65 a 70': 8, '70 a 75': 9, '< 1': 10, '> 75': 11}
Nombre de la entidad: {'ALIANSALUD EPS S.A.': 0, 'ALIANZA MEDELLIN ANTIOQUIA EPS S.A.S. "SAVIA SALUD EPS" -CM': 1, 'ASMET SALUD EPS S.A.S. -CM': 2, 'ASOCIACION MUTUAL SER EMPRESA SOLIDARIA DE SALUD ENTIDAD PROMOTORA DE SALUD - MUTUAL SER EPS': 3, 'ASOCIACION MUTUAL SER EMPRESA SOLIDARIA DE SALUD ENTIDAD PROMOTORA DE SALUD - MUTUAL SER EPS -CM': 4, 'ASOCIACIÓN DE CABILDOS INDÍGENAS DEL CESAR Y GUAJIRA "DUSAKAWI A.R.S.I." -CM': 5, 'ASOCIACIÓN INDÍGENA DEL CAUCA A.I.C. EPSI -CM': 6, 'CAJA DE COMPENSACION FAMILIAR DEL VALLE DEL CAUCA "COMFENALCO VALLE DE LA GENTE': 7, 'CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR': 8, 'CAJA DE COMPENSACIÓN FAMILIAR DEL CHOCÓ -CM': 9, 'CAJA DE COMPENSACIÓN FAMILIAR DEL ORIENTE COLOMBIANO "COMFAORIENTE" -CM': 10, 'CAJACOPI EPS S.A.S -CM': 11, 'CA

In [44]:
datos_codificados.head()

,Tipo de afiliado_ADICIONAL,Tipo de afiliado_BENEFICIARIO,Tipo de afiliado_COTIZANTE,Estado del afiliado_Activo,Estado del afiliado_Protección Laboral C,Condición del beneficiario_DISCAPACIDAD,Condición del beneficiario_ESTUDIANTE,Condición del beneficiario_NO APLICA,Zona de Afiliación_Rural,Zona de Afiliación_Urbana,Género_Femenino,Género_Masculino,Grupo etario_codificado,Nombre de la entidad_codificado,Departamento_codificado,tps_nvl_ssb_id_codificado
0,0,0,1,1,0,0,0,1,1,0,0,1,2,2,10,2
1,0,0,1,1,0,0,0,1,0,1,1,0,5,23,29,3
2,0,1,0,1,0,0,0,1,0,1,1,0,4,15,1,3
3,0,0,1,1,0,0,0,1,0,1,1,0,1,20,10,3
4,0,1,0,1,0,0,0,1,0,1,0,1,9,23,14,3


<br></br>
*Exploración Final*

In [13]:
print(datos_codificados.shape)
print( )
print(datos_codificados.info())

(305247, 16)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305247 entries, 0 to 305246
Data columns (total 16 columns):
 #   Column                                    Non-Null Count   Dtype
---  ------                                    --------------   -----
 0   Tipo de afiliado_ADICIONAL                305247 non-null  int64
 1   Tipo de afiliado_BENEFICIARIO             305247 non-null  int64
 2   Tipo de afiliado_COTIZANTE                305247 non-null  int64
 3   Estado del afiliado_Activo                305247 non-null  int64
 4   Estado del afiliado_Protección Laboral C  305247 non-null  int64
 5   Condición del beneficiario_DISCAPACIDAD   305247 non-null  int64
 6   Condición del beneficiario_ESTUDIANTE     305247 non-null  int64
 7   Condición del beneficiario_NO APLICA      305247 non-null  int64
 8   Zona de Afiliación_Rural                  305247 non-null  int64
 9   Zona de Afiliación_Urbana                 305247 non-null  int64
 10  Género_Femenino               

In [14]:
porcentaje_nulos = (datos_codificados.isnull().sum() / len(datos_codificados)) * 100
print(porcentaje_nulos)

Tipo de afiliado_ADICIONAL                  0.0
Tipo de afiliado_BENEFICIARIO               0.0
Tipo de afiliado_COTIZANTE                  0.0
Estado del afiliado_Activo                  0.0
Estado del afiliado_Protección Laboral C    0.0
Condición del beneficiario_DISCAPACIDAD     0.0
Condición del beneficiario_ESTUDIANTE       0.0
Condición del beneficiario_NO APLICA        0.0
Zona de Afiliación_Rural                    0.0
Zona de Afiliación_Urbana                   0.0
Género_Femenino                             0.0
Género_Masculino                            0.0
Grupo etario_codificado                     0.0
Nombre de la entidad_codificado             0.0
Departamento_codificado                     0.0
tps_nvl_ssb_id_codificado                   0.0
dtype: float64


Es importante aclarar que aquí se hizo una imputación y codificación a la vez ya que para los datos tipo nan, se les asignó también un valor. Es decir, los NaNs, también entrarán dentro de la clasificación.

In [17]:
columna_a_filtrar = 'tps_nvl_ssb_id_codificado'

df_filtrado = datos_codificados[datos_codificados[columna_a_filtrar] != 3]

df_filtrado.head()


DataFrame filtrado:


,Tipo de afiliado_ADICIONAL,Tipo de afiliado_BENEFICIARIO,Tipo de afiliado_COTIZANTE,Estado del afiliado_Activo,Estado del afiliado_Protección Laboral C,Condición del beneficiario_DISCAPACIDAD,Condición del beneficiario_ESTUDIANTE,Condición del beneficiario_NO APLICA,Zona de Afiliación_Rural,Zona de Afiliación_Urbana,Género_Femenino,Género_Masculino,Grupo etario_codificado,Nombre de la entidad_codificado,Departamento_codificado,tps_nvl_ssb_id_codificado
0,0,0,1,1,0,0,0,1,1,0,0,1,2,2,10,2
13,0,0,1,1,0,0,0,1,1,0,0,1,2,20,4,1
17,0,0,1,1,0,0,0,1,0,1,1,0,2,16,18,0
31,0,0,1,1,0,0,0,1,1,0,1,0,3,18,30,0
35,0,0,1,1,0,0,0,1,0,1,0,1,3,29,29,2


In [58]:
conteo_SURA = df_filtrado['Nombre de la entidad_codificado'].value_counts().get(24, 0)
conteo_SURA


0

Como conclusión se obtiene que no se pueden borrar los datos faltantes de SISBÉN ya que desaparecen todos los afiliados a EPS SURA por lo que dañaría todo el proyecto ya que esos son los datos con los que queremos entrenar, así que se borrarán ambas columnas del sisbén por tener tantos datos faltanes y se hará la clasificación con las otras columnas. 

In [36]:
columnas_Eliminar = ['tps_nvl_ssb_id_codificado']

df_listo = datos_codificados.drop(columns=columnas_Eliminar)
df_listo.head()

,Tipo de afiliado_ADICIONAL,Tipo de afiliado_BENEFICIARIO,Tipo de afiliado_COTIZANTE,Estado del afiliado_Activo,Estado del afiliado_Protección Laboral C,Condición del beneficiario_DISCAPACIDAD,Condición del beneficiario_ESTUDIANTE,Condición del beneficiario_NO APLICA,Zona de Afiliación_Rural,Zona de Afiliación_Urbana,Género_Femenino,Género_Masculino,Grupo etario_codificado,Nombre de la entidad_codificado,Departamento_codificado
0,0,0,1,1,0,0,0,1,1,0,0,1,2,2,10
1,0,0,1,1,0,0,0,1,0,1,1,0,5,23,29
2,0,1,0,1,0,0,0,1,0,1,1,0,4,15,1
3,0,0,1,1,0,0,0,1,0,1,1,0,1,20,10
4,0,1,0,1,0,0,0,1,0,1,0,1,9,23,14


In [60]:
conteo_SURA = df_listo['Nombre de la entidad_codificado'].value_counts().get(24, 0)
conteo_SURA

11073

In [62]:
df_listo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305247 entries, 0 to 305246
Data columns (total 15 columns):
 #   Column                                    Non-Null Count   Dtype
---  ------                                    --------------   -----
 0   Tipo de afiliado_ADICIONAL                305247 non-null  int64
 1   Tipo de afiliado_BENEFICIARIO             305247 non-null  int64
 2   Tipo de afiliado_COTIZANTE                305247 non-null  int64
 3   Estado del afiliado_Activo                305247 non-null  int64
 4   Estado del afiliado_Protección Laboral C  305247 non-null  int64
 5   Condición del beneficiario_DISCAPACIDAD   305247 non-null  int64
 6   Condición del beneficiario_ESTUDIANTE     305247 non-null  int64
 7   Condición del beneficiario_NO APLICA      305247 non-null  int64
 8   Zona de Afiliación_Rural                  305247 non-null  int64
 9   Zona de Afiliación_Urbana                 305247 non-null  int64
 10  Género_Femenino                           30